In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4' # nvidia-smi로 비어있는 gpu 확인하고 여기서 선택할것!

from transformers import (
    PreTrainedTokenizerFast as BaseGPT2Tokenizer,
    EncoderDecoderModel,
    DataCollatorForSeq2Seq,
)


from asset.tokenization_kobert import KoBertTokenizer
from asset import tokenization_kobert
src_tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

#from huggingface_hub import notebook_login

#notebook_login()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [2]:
class GPT2Tokenizer(BaseGPT2Tokenizer):
    def build_inputs_with_special_tokens(self, token_ids, _):
        return token_ids + [self.eos_token_id]
trg_tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

In [3]:
model = EncoderDecoderModel.from_pretrained('./dacon-v0/checkpoint-36000')
model.eval()
model.config.decoder_start_token_id = trg_tokenizer.bos_token_id

In [4]:
import numpy as np
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2
import pandas as pd

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
smodel = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [5]:
# 문장 예시
preds = [
    "이번 경진대회는 질의 응답 처리를 수행하는 AI 모델을 개발해야합니다.",
    "데이콘은 플랫폼입니다."
]

gts = [
    "이번 경진대회의 주제는 도배 하자 질의 응답 AI 모델 개발입니다.",
    "데이콘은 국내 최대의 AI 경진대회 플랫폼입니다."
]

In [6]:
# 샘플에 대한 Cosine Similarity 산식
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

In [7]:
sample_scores = []
for pred, gt in zip(preds, gts):
    # 생성된 답변 내용을 512 Embedding Vector로 변환
    pred_embed = smodel.encode(pred)
    gt_embed = smodel.encode(gt)
    
    sample_score = cosine_similarity(gt_embed, pred_embed)
    # Cosine Similarity Score가 0보다 작으면 0으로 간주
    sample_score = max(sample_score, 0)
    print('예측 : ', pred)
    print('정답 : ', gt)
    print('Cosine Similarity Score : ', sample_score)
    print('-'*20)
    sample_scores.append(sample_score)
print('전체 샘플의 Cosine Similarity Score 평균 : ', np.mean(sample_scores))

예측 :  이번 경진대회는 질의 응답 처리를 수행하는 AI 모델을 개발해야합니다.
정답 :  이번 경진대회의 주제는 도배 하자 질의 응답 AI 모델 개발입니다.
Cosine Similarity Score :  0.83436465
--------------------
예측 :  데이콘은 플랫폼입니다.
정답 :  데이콘은 국내 최대의 AI 경진대회 플랫폼입니다.
Cosine Similarity Score :  0.6281873
--------------------
전체 샘플의 Cosine Similarity Score 평균 :  0.731276


In [8]:
input_list = []


data = pd.read_csv("./bigdata/train.csv")
input_dict = {"utterance":[]}
for _, row in data.iterrows():
    for q_col in ['질문_1', '질문_2']:
        for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
            input_list.append({"question":row[q_col],"answer":row[a_col]})
            


In [9]:
def read_input(path):
    test_question = []
    test = pd.read_csv(path)
    for ut in test['질문']:
        test_question.append(ut)

    return test_question

In [10]:
test_q= read_input('./bigdata/test.csv')

In [11]:
gt_q_embed = []

for gt_q_dic in input_list:
    gt_q_embed.append({"question_embed":smodel.encode(gt_q_dic["question"]),"question":gt_q_dic["question"],"answer":gt_q_dic["answer"]})
    


In [12]:


def translate(text):
    max_sim_dic = {"question_embed":None,"question":None,"answer":None}
    max_sim = -100
    text_embed = smodel.encode(text)
    for gt_dic in gt_q_embed:
        tmp_sim = cosine_similarity(gt_dic["question_embed"], text_embed)
        if tmp_sim > max_sim:
            if "장점" or "단점" in text:
                if ("장점" in text and "단점" in gt_dic["question"]) or ("단점" in text and "장점" in gt_dic["question"]):
                    #print("there was an error")
                    continue
            max_sim = tmp_sim
            max_sim_dic = {"question_embed":gt,"question":gt_dic["question"],"answer":gt_dic["answer"]}
    
    
    if max_sim < 0.9:
        
        embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
        embeddings = {k: v for k, v in embeddings.items()}
        # generate 함수에 하이퍼파라미터 추가
        output_sequences = model.generate(
            **embeddings,
            max_length=65,
            temperature=0.9,
            top_k=1,
            top_p=0.9,
            repetition_penalty=1.2,
            do_sample=True,
            num_return_sequences=1
        )
    
        tmp = trg_tokenizer.decode(output_sequences[0, 1:-1].cpu())
        return tmp
    
    #print("max_sim is...  " +str(max_sim))
    return max_sim_dic["answer"]

In [13]:
def translate_framework(text):
    pred = ""
    dividers = ["또한,","그리고"]
    special_divider1 = ["와 ","관계는 무엇인가요?"]
    for divider in dividers:
        
        if divider in text:
            text1 = text.split(divider)[0].strip()
            text2 = text.split(divider)[1].strip()
            pred = translate(text1) +" "+divider+" "+ translate(text2)
            return pred
    
    
    
    if special_divider1[1] in text:
        text1 = text.split(special_divider1[0])[0].strip()
        text2 = text.split(special_divider1[0])[1].strip()[:-12]
        pred = translate(text1) +" "+divider+" "+ translate(text2)
        return pred
    
    pred = translate(text)
    return pred
    

In [14]:
preds = []
for i in range(len(test_q)):
    sr_text = test_q[i] # dialect
    tg_text = translate_framework(sr_text) # translate
    preds.append(tg_text)
    if i<30:
        print("source_text : ",sr_text)
        print("result :      ",tg_text)
        print()
        print()

/home/leadawon5/gitfiles/venvs/myvenv/lib/python3.7/site-packages/transformers/generation/utils.py:1260: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  "You have modified the pretrained model configuration to control generation. This is a"


source_text :  방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
result :       방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 이러한 페인트들은 각각의 특성과 용도에 맞게 선택하여 사용할 수 있습니다. 광명 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.


source_text :  도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
result :       도배지에 얼룩이 발생하는 경우, 해당 부분을 전문적인 도구와 화학약품을 사용하여 철저히 제거해야 합니다. 먼저 누수를 일으킨 원인을 파악하고 수정한 후, 전문가의 도움을 받아 도배지를 청소하고 건조시키는 것이 좋습니다. 만약에도 도배지가 젖어서 곰팡이가 발견되면 전문가의 도움을 받아야 할 수도 있습니다. 따라서 도배작업 시에는


source_text :  큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
result :       큐블럭은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점이 단점입니다. 또한 설치 시 추가적으로 접착재와 전문 기술이 요구되어 비용이 더 들어갈 수 있다는 점도 단점으로 꼽힙니다. 이러한 점들을 감안하여 건축자 및 시공 업체들은 클라이언트에게 건축 또한, 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간편하다는 장점이 있습니다.


source_text :  철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있을까요?
result :       철골구조는 화재 발생 시 내구성이 약해지는 특성이 있습니다. 그러나 이를 보완하

In [15]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(preds)
pred_embeddings.shape

(130, 512)

In [16]:
submit = pd.read_csv('./bigdata/sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.020689,0.000008,-0.001449,0.006100,0.064982,0.020654,0.008956,0.006110,-0.017802,...,0.003074,-0.029102,-0.019838,-0.027554,-0.014815,0.008810,0.031672,0.021500,-0.000919,0.018775
1,TEST_001,0.025184,0.041127,-0.008241,0.002660,0.115862,0.000004,0.076680,0.010812,0.003022,...,-0.021348,0.025135,0.026205,-0.014108,-0.016448,0.019633,0.039662,0.010611,0.020869,0.023393
2,TEST_002,-0.000261,-0.049852,-0.011959,0.012029,0.110353,-0.046253,-0.007455,-0.014188,0.039247,...,-0.007456,-0.033012,0.059268,-0.004583,-0.017719,0.006105,-0.011327,-0.023792,-0.026269,0.078509
3,TEST_003,-0.014142,0.011043,-0.017837,0.012748,0.101084,-0.025850,-0.077918,0.013456,0.009481,...,-0.033575,-0.002281,0.062397,-0.051405,0.002629,0.025059,0.005195,-0.007075,-0.048920,0.036650
4,TEST_004,0.000527,0.017190,-0.007151,-0.011216,0.112582,0.018102,0.052409,0.025979,-0.022784,...,-0.011692,-0.003198,0.021746,0.011912,-0.030977,-0.004547,0.002966,0.032793,0.003943,0.044688


In [17]:
# 리더보드 제출을 위한 csv파일 생성
submit.to_csv('./submit/dacon-v0_submit.csv', index=False)